# Phys555 Assignment 2 Question 1
Karlee Zammit - V00823093

## Datasets, Data Mining, and Data Visualization (using PCA)

Q1- Find two 'suitable' and 'challenging' data sets; one for regression and another for a classification problem. Describe the two data sets in one notebook (inputs and targets) and explain how they can be used for regression and classification. Write about works/applications that have been previously done regarding the chosen data sets. Also, conduct data mining, such as feature visualization and present informative data, using PCA.

## References



https://www.kaggle.com/datasets/sohier/calcofi?resource=download

https://towardsdatascience.com/linear-regression-modelling-oceanographic-data-fdc3a98813bf

https://www.kaggle.com/datasets/vencerlanz09/sea-animals-image-dataste